# Параллельная разработка с библиотекой `asyncio`

## Основы

Здесь сначала введем новый термин - конкурентность (от англ. concurrency). Почти везде слово concurrency на русский переводится как параллелизм, однако это не совсем корректный перевод. Параллелизм - это когда одновременно работают несколько потоков операций. Допустим, у нас есть ПК с 4 ядрами ЦПУ. Тогда параллельно могут работать не более 4 потоков. При этом, если посмотреть на диспетчер задач, то там скорее всего будет числиться гораздо большее число одновременно выполняемых процессов. То есть, операционная система выделяет каждому из потоков этих процессов какой-то квант процессорного времени, чередуя потоки. Это и есть пример конкурентности. 

В отличие от `concurrent.futures` библиотека `asyncio` реализует многозадачность через конкурентность с помощью корутин и использования цикла обработки событий. Начнем наше знакомство с `asyncio` с изучения различий с явной работой с потоками.

In [4]:
%%writefile spinner_thread.py

import threading
import time
import sys
import itertools

class Signal:
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = f'{char} {msg}'
        write(status)
        flush()
        write('x\08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + 'x\08' * len(status))
    
def slow_function():
    time.sleep(3)
    return 42
    
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result
    
def main():
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

Overwriting spinner_thread.py
